In [1]:
# import os
# from neo4j import GraphDatabase as graphdb
# from trengx import graph

import os
from dotenv import load_dotenv
from neo4j import GraphDatabase as graphdb
import sys
sys.path.append('../trengx')
import graph

In [2]:
# .env secrets
uri = os.getenv('NEO4J_URI') # Get the value of the uri variable
user = os.getenv('NEO4J_USER') # Get the value of the user variable
password= os.getenv('NEO4J_PASSWORD')  # Get the value of the pw variable

# Create a Graph class instance
g = graph.Graph(graphdb, uri, user, password)


In [3]:
g.delete_all()

# y_est[i] = w_est[i]*y[i-1]
w_est_i = g.add_node ('num', 'w_est_i', {'value': 0.1})
w_est_i_id = w_est_i['node_id']
y_im1 = g.add_node ('num', 'y_im1 ', {'value': 3.0})
y_im1_id = y_im1['node_id']
mul = g.add_node ('op', '*', {'reverse':False})
mul_id = mul['node_id']
print (mul_id)
g.add_edge ('num2op', w_est_i_id, mul_id, {'trigger':False})
g.add_edge ('num2op', y_im1_id, mul_id, {'trigger':True})
y_est_i = g.add_node ('num', 'y_est_i', {'value': 2.0})
y_est_i_id = y_est_i['node_id']
print (y_est_i_id)
g.add_edge ('op2num', mul_id, y_est_i_id, {'trigger':True})

g.set_node_prop(y_im1_id, 'value', 20)
result = g.get_node_prop(y_est_i_id, 'value')
print (result)
g.close()

12
13


In [25]:
g.close()